In [41]:
import torch
import sys
import os
from pathlib import Path
import numpy as np
from typing import List
from PIL import Image
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

sys.path.append(".")
sys.path.append("..")
sys.path.append(os.path.abspath('D:/Docs/Kursach/stylegan3-editing/utils/'))

from models.stylegan3.model import SG3Generator
from common import make_transform, tensor2im
from notebooks.notebook_utils import run_alignment, crop_image, compute_transforms

In [42]:
generator = SG3Generator(checkpoint_path="../pretrained_models/stylegan3-r-ffhq-1024x1024.pkl").decoder

num_images_per_cluster = 5
trunc = 0.5

INTERFACEGAN_RANGES = {
    "asian_eyes": (-4, 5),
    "age": (-5, 5),
    "smile": (-2, 2),
    "Male": (-2, 4)
}

Loading StyleGAN3 generator from path: ../pretrained_models/stylegan3-r-ffhq-1024x1024.pkl
Done!


In [43]:
csv_name = 'sonar_all-data'
df = pd.read_csv(f"{Path('./datasets/') / csv_name}.csv")
df = df.select_dtypes(exclude=['object'])
custom_images = ["gladkov.jpg", "gorodilov.jpg", "smith.jpg"]

In [44]:
aligned_img = []
for image_path in custom_images:
    img_p = Path('./images/') / image_path
    input_image = run_alignment(img_p)
    joined = np.concatenate([input_image.resize((256, 256))], axis=1)
    aligned_img.append(Image.fromarray(joined))

Done.
Aligning image...
Finished aligning image: images\gladkov.jpg
Done.
Aligning image...
Finished aligning image: images\gorodilov.jpg
Done.
Aligning image...
Finished aligning image: images\smith.jpg


In [45]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df.iloc[:])

In [46]:
num_samples, num_atributes = df_scaled.shape
df_latents = np.pad(df_scaled, ((0, 0), (0, 512-num_atributes)))
kmeans = KMeans(n_clusters=3, random_state=0).fit(df_latents)
cluster_assignments = kmeans.predict(df_latents)

In [ ]:
cluster_images = []
for latent in df_latents:
    z = torch.from_numpy(latent).float().to('cuda').unsqueeze(0)
    w = generator.mapping(z, None, truncation_psi=trunc)
    with torch.no_grad():
        img = generator.synthesis(w, noise_mode='const')
        cluster_images.append(tensor2im(img[0]))

start_path = Path(f'./edit_results/{csv_name}')
start_path.mkdir(exist_ok=True, parents=True)
for i, img in enumerate(cluster_images):
    img.save(start_path / f"data_in_image_{i+1}.png")